# LaTok

## Linear Algebraic Tokenizer

## Main Idea and Motivation

* Tokenization performance is critical
    * Many NLP tasks require tokenization of large corpora
        * Poor tokenization performance is a processing bottleneck
    * There are a lot of tokenizers with varying
        * behavior
        * configurability
        * performance
* NumPy is fast
    * In practice, existing NumPy functions often outperform raw python implementations and often even Cython optimizations.


### Question: Could numpy.split be a basis for a fast, configurable tokenizer?
    

## Tokenizing using NumPy Split

For a **text** string, mark the positions of "split" points with a **mask**

For example, to split on spaces, align mask 1's with the text's spaces:

In [1]:
text = 'This is a test'
mask = '00001001010000'

Turn these into alignable numpy arrays suitable for applying the "split" function:

In [2]:
import numpy as np

ntext = np.array(list(text))
nmask = np.array(list(mask), dtype=np.int8)
print(list(zip(ntext, nmask)))

[('T', 0), ('h', 0), ('i', 0), ('s', 0), (' ', 1), ('i', 0), ('s', 0), (' ', 1), ('a', 0), (' ', 1), ('t', 0), ('e', 0), ('s', 0), ('t', 0)]


See what the split looks like:

In [3]:
np.split(ntext, np.flatnonzero(nmask))

[array(['T', 'h', 'i', 's'], dtype='<U1'),
 array([' ', 'i', 's'], dtype='<U1'),
 array([' ', 'a'], dtype='<U1'),
 array([' ', 't', 'e', 's', 't'], dtype='<U1')]

And convert back to tokens:

In [4]:
[''.join(a).strip() for a in np.split(ntext, np.flatnonzero(nmask))]

['This', 'is', 'a', 'test']

#### Hey, this might be crazy enough to actually work!

## Tokenization via Linear Operations on a Feature Matrix

1. Define $\mathbf{m}$ character features
    * e.g., isspace, isalpha, etc.
2. Vectorize the $\mathbf{n}$ text characters into an $\mathbf{n}$x$\mathbf{m}$ feature matrix, $\mathbf{F}$, where each element $\mathbf{f_{i,j}}$ is the $\mathbf{j^{th}}$ feature for the $\mathbf{i^{th}}$ character.
    * One row for each text character
    * Each column represents a character feature

$$ F = \begin{bmatrix} f_{1,1} & f_{1,2} & \ldots & f_{2,m} \\ f_{2,1} & f_{2,2} & \ldots & f_{2,m} \\ \ldots & \ldots & \ldots & \ldots \\ f_{n,1} & f_{n,2} & \ldots & f_{n,m} \end{bmatrix} $$

3. Define a linear transformation, $\mathbf{S}$, on $\mathbf{F}$ to produce an $\mathbf{n}$-dimensional split mask vector, $\mathbf{s}$,
    * Where vector values are non-zero at character positions at which to split and consecutively zero to identify spans of characters to be combined as tokens.

$$ \begin{equation*} \mathbf{S} ( \mathbf{F} ) = \mathbf{s} \end{equation*} $$

so that

$$ s = \begin{bmatrix} s_{1} \\ s_{2} \\ \ldots \\ s_{n} \end{bmatrix} $$

where

$$ \begin{align} \mathbf{s_{i}} \neq 0, \quad \quad \text{to start a token at position i}, \\ \text{and} \quad \mathbf{s_{j}} = 0, \quad \quad \text{for characters within tokens} \end{align} $$

4. Apply the split mask to tokenize the text string


...Next, let's pursue this idea with a concrete example...